# IPL Betting Models: Data Engineering

### Using the existing dataset to engineer better features


In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Loading the datasets

df = pd.read_csv('~/code/patrickevans29/raw_data/complete_cleaned_dataset.csv')
ball_df = pd.read_csv('~/code/patrickevans29/raw_data/IPL_Ball_by_Ball_2008_2022_cleaned.csv')

# 1. Batsman

### Using the ball by ball data to extract key information for each batsman

In [4]:
# Exploring the data

ball_df.head()

,ID,innings,overs,ballnumber,batter,bowler,non_striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,1,1,1,ybk jaiswal,mohammed shami,jc buttler,noextra,0,0,0,0,0,noplayerout,nokind,nofieldersinvolved,rajasthan royals
1,1312200,1,1,2,ybk jaiswal,mohammed shami,jc buttler,legbyes,0,1,1,0,0,noplayerout,nokind,nofieldersinvolved,rajasthan royals
2,1312200,1,1,3,jc buttler,mohammed shami,ybk jaiswal,noextra,1,0,1,0,0,noplayerout,nokind,nofieldersinvolved,rajasthan royals
3,1312200,1,1,4,ybk jaiswal,mohammed shami,jc buttler,noextra,0,0,0,0,0,noplayerout,nokind,nofieldersinvolved,rajasthan royals
4,1312200,1,1,5,ybk jaiswal,mohammed shami,jc buttler,noextra,0,0,0,0,0,noplayerout,nokind,nofieldersinvolved,rajasthan royals


In [5]:
# Calculating the total runs scored by each batsman

batter_total_runs = ball_df.groupby(['batter'], as_index=False)['batsman_run'].sum('batsman_run')\
                        .rename(columns={'batsman_run': 'total_runs'})

In [6]:
# Calculating the balls faced by each batsman

balls_faced = ball_df[ball_df['extra_type'].isin(['noextra', 'legbyes', 'byes', 'noballs'])]\
                .groupby(['batter'], as_index=False)['ballnumber'].count()\
                .rename(columns={'ballnumber': 'balls_faced'})

In [7]:
# Calculating the number of innings for each batsman

batter_innings = ball_df.groupby(['batter'], as_index=False)['ID'].nunique()\
                    .rename(columns={'ID': 'batter_innings'})
                    
# Updating for non-strikers

non_striker_innings = ball_df.groupby(['non_striker'], as_index=False)['ID'].nunique()\
                    .rename(columns={'ID': 'non_striker_innings', 'non_striker': 'batter'})
                    
combined_innings_df = batter_innings.merge(non_striker_innings, on='batter', how='outer').fillna(0)

combined_innings_df['bat_innings'] = combined_innings_df[['batter_innings', 'non_striker_innings']].max(axis=1)

# Final dataframe

bat_innings = combined_innings_df.drop(columns=['batter_innings', 'non_striker_innings'])
                    


In [8]:
## Calculating the boundaries scored by each batsman
# A number of fours / sixes appear as non-boundary in the dataset

non_boundary = list(ball_df[ball_df['non_boundary'] == 1].index)

# Creating a temporary df to remove the non_boundary indexes

temp_df = ball_df.drop(index=non_boundary)

## Calculating the fours and sixes scored by each batsman

fours = temp_df[temp_df['batsman_run'] == 4].groupby(['batter'], as_index=False)\
    ['batsman_run'].count().rename(columns={'batsman_run': 'fours'})
    
sixes = temp_df[temp_df['batsman_run'] == 6].groupby(['batter'], as_index=False)\
    ['batsman_run'].count().rename(columns={'batsman_run': 'sixes'})

In [9]:
## Calculating the 0, 50 and 100 totals for each batsman

# Creating a batsman score for each game ID

batsman_score_game = ball_df.groupby(['ID', 'batter'], as_index=False)['batsman_run'].sum().drop(columns='ID')

# Filter the new dataframe to get 0, 50 and 100 scores only

zero = batsman_score_game[batsman_score_game['batsman_run'] == 0].groupby(['batter'], as_index=False)\
        .count().rename(columns={'batsman_run': 'zero'})

fifty = batsman_score_game[batsman_score_game['batsman_run'] >= 50].groupby(['batter'], as_index=False)\
        .count().rename(columns={'batsman_run': '50s'})
        
hundred = batsman_score_game[batsman_score_game['batsman_run'] >= 100].groupby(['batter'], as_index=False)\
        .count().rename(columns={'batsman_run': '100s'})
        
# Combining and removing the double counts

merged = zero.merge(fifty, on='batter', how='outer').merge(hundred, on='batter', how='outer').fillna(0)
merged['zero'] = merged['zero'].astype('int')
merged['50s'] = merged['50s'].sub(merged['100s']).astype('int')
merged['100s'] = merged['100s'].astype('int')

In [10]:
## Calculating batsman not out at innings end

# Finding the index of the last ball of each inning

last_ball_index_temp = list(ball_df.groupby(['ID', 'innings'], as_index=False)['innings']\
                        .idxmax().sort_values(by='innings', ascending=False)['innings'])

last_ball_index_temp.remove(0)

last_ball_index = [index - 1 for index in last_ball_index_temp]

last_ball_index.insert(0, ball_df.shape[0])

# Counting the player out on the final ball

player_out_final_ball = ball_df[(ball_df['isWicketDelivery']==1) & (ball_df.index.isin(last_ball_index))]\
                        .groupby(['player_out'], as_index=False)['ID'].count()\
                        .rename(columns={'ID': 'out_count', 'player_out': 'batter'})
                        
# Counting all batsman involved in final ball of an inning

final_ball_batter = ball_df[ball_df.index.isin(last_ball_index)].groupby(['batter'], as_index=False)\
                    ['ID'].count().rename(columns={'ID': 'at_bat_count'})
                    
final_ball_non_striker = ball_df[ball_df.index.isin(last_ball_index)].groupby(['non_striker'], as_index=False)\
                        ['ID'].count().rename(columns={'ID': 'non_striker_count', 'non_striker': 'batter'})
                        
# Combining the dataframes and calculating not_out for each batsman

combined_final_ball = final_ball_batter.merge(final_ball_non_striker, on='batter', how='outer')\
                        .merge(player_out_final_ball, on='batter', how='outer')
                        
combined_final_ball.fillna(0, inplace=True)

combined_final_ball['not_out'] = combined_final_ball['at_bat_count']\
                                .add(combined_final_ball['non_striker_count'])\
                                .sub(combined_final_ball['out_count']).astype('int')
                                
# Final dataframe

not_out = combined_final_ball[['batter', 'not_out']]


In [11]:
# Calculate the high score for each batsman

high_score = ball_df.groupby(['ID', 'batter'], as_index=False)['batsman_run']\
                        .sum().groupby('batter', as_index=False)['batsman_run'].max()\
                        .rename(columns={'batsman_run': 'high_score'})

In [12]:
# Merge all the statistics extracted

batting_stats_merged = bat_innings.merge(batter_total_runs, on='batter', how='outer')\
                        .merge(not_out, on='batter', how='outer')\
                        .merge(merged, on='batter', how='outer')\
                        .merge(fours, on='batter', how='outer')\
                        .merge(sixes, on='batter', how='outer')\
                        .merge(high_score, on='batter', how='outer')\
                        .merge(balls_faced, on='batter', how='outer').fillna(0)

In [13]:
# Check the dataframe

batting_stats_merged

,batter,bat_innings,total_runs,not_out,zero,50s,100s,fours,sixes,high_score,balls_faced
0,a ashish reddy,23.0,280.0,8.0,0.0,0.0,0.0,16.0,15.0,36.0,193.0
1,a badoni,11.0,161.0,3.0,1.0,1.0,0.0,11.0,7.0,54.0,130.0
2,a chandila,2.0,4.0,2.0,1.0,0.0,0.0,0.0,0.0,4.0,7.0
3,a chopra,6.0,53.0,0.0,0.0,0.0,0.0,7.0,0.0,24.0,71.0
4,a choudhary,3.0,25.0,2.0,0.0,0.0,0.0,1.0,1.0,15.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...
604,z khan,31.0,117.0,18.0,6.0,0.0,0.0,11.0,2.0,23.0,141.0
605,a nel,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
606,bw hilfenhaus,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,c ganapathy,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Calculate average score - total_runs / bat_innings

batting_stats_merged['average_score'] = round((batting_stats_merged['total_runs'] / batting_stats_merged['bat_innings']), 2).fillna(0)

In [15]:
# Calculate batting average - total_runs / (bat_innings - not_out)

batting_stats_merged['batting_average'] = round((batting_stats_merged['total_runs'] / \
    (batting_stats_merged['bat_innings'] - batting_stats_merged['not_out'])), 2).fillna(0)

# Replace infinite values with the average score

batting_stats_merged['batting_average'] = np.where(batting_stats_merged\
    ['batting_average'] == np.inf, batting_stats_merged['average_score'], batting_stats_merged['batting_average'])

In [16]:
# Calculate batting strike rate - (total_runs * 100) / balls_faced

batting_stats_merged['batting_strike_rate'] = round((batting_stats_merged['total_runs'] * 100)/ \
                                                batting_stats_merged['balls_faced'], 2).fillna(0)

In [17]:
# Final dataframe

batting_stats_merged

,batter,bat_innings,total_runs,not_out,zero,50s,100s,fours,sixes,high_score,balls_faced,average_score,batting_average,batting_strike_rate
0,a ashish reddy,23.0,280.0,8.0,0.0,0.0,0.0,16.0,15.0,36.0,193.0,12.17,18.67,145.08
1,a badoni,11.0,161.0,3.0,1.0,1.0,0.0,11.0,7.0,54.0,130.0,14.64,20.12,123.85
2,a chandila,2.0,4.0,2.0,1.0,0.0,0.0,0.0,0.0,4.0,7.0,2.00,2.00,57.14
3,a chopra,6.0,53.0,0.0,0.0,0.0,0.0,7.0,0.0,24.0,71.0,8.83,8.83,74.65
4,a choudhary,3.0,25.0,2.0,0.0,0.0,0.0,1.0,1.0,15.0,20.0,8.33,25.00,125.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,z khan,31.0,117.0,18.0,6.0,0.0,0.0,11.0,2.0,23.0,141.0,3.77,9.00,82.98
605,a nel,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
606,bw hilfenhaus,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
607,c ganapathy,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00


In [18]:
# Export

# batting_stats_merged.to_csv('~/code/patrickevans29/raw_data/batting_features_dataset.csv', index=False)

# 2. Bowlers

### Using the ball by ball data to extract key information for each bowler

In [29]:
ball_df.tail(2)

,ID,innings,overs,ballnumber,batter,bowler,non_striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
225952,335982,2,16,1,sb joshi,lr shukla,p kumar,wides,0,1,1,0,0,noplayerout,nokind,nofieldersinvolved,royal challengers bangalore
225953,335982,2,16,2,sb joshi,lr shukla,p kumar,noextra,0,0,0,0,1,sb joshi,caught,bb mccullum,royal challengers bangalore


In [21]:
# Calculating the number of innings for each bowler

bowler_innings = ball_df.groupby(['bowler'], as_index=False)['ID'].nunique()\
                    .rename(columns={'ID': 'bowler_innings'})

In [59]:
# Calculating the number of balls bowled by each bowler

balls_bowled = ball_df[ball_df['extra_type'].isin(['noextra', 'legbyes', 'byes', 'penalty'])]\
                .groupby(['bowler'], as_index=False)['ballnumber'].count()\
                .rename(columns={'ballnumber': 'balls_bowled'})

In [57]:
# Calculating the number of runs conceded by each bowler

runs_conceded = ball_df[ball_df['extra_type'].isin(['noextra', 'wides', 'noballs'])]\
                .groupby(['bowler'], as_index=False)['total_run'].sum()\
                .rename(columns={'total_run': 'runs_conceded'})

In [69]:
# Calculating the number of wickets taken by each bowler

wickets = ball_df[(ball_df['isWicketDelivery'] == 1) & (ball_df['kind'].isin\
                (['nokind', 'caught', 'caught and bowled', 'bowled','stumped', 'lbw', 'hit wicket']))]\
                .groupby(['bowler'], as_index=False)['isWicketDelivery'].count()\
                .rename(columns={'isWicketDelivery': 'wickets'})
                


In [111]:
# Calculating the wicket stats for each bowler

wicket_count_per_game = ball_df[(ball_df['isWicketDelivery'] == 1) & (ball_df['kind'].isin\
                (['nokind', 'caught', 'caught and bowled', 'bowled','stumped', 'lbw', 'hit wicket']))]\
                .groupby(['ID', 'bowler'], as_index=False)['isWicketDelivery'].count()\
                .rename(columns={'isWicketDelivery': 'wickets'})
                
four_wickets = wicket_count_per_game[wicket_count_per_game['wickets'] == 4]\
                .groupby(['bowler'], as_index=False)['wickets'].count()\
                .rename(columns={'wickets': 'four_wickets'})
                
five_wickets = wicket_count_per_game[wicket_count_per_game['wickets'] == 5]\
                .groupby(['bowler'], as_index=False)['wickets'].count()\
                .rename(columns={'wickets': 'five_wickets'})

In [112]:
# Merge all the statistics extracted

bowling_stats_merged = bowler_innings.merge(balls_bowled, on='bowler', how='outer')\
                        .merge(runs_conceded, on='bowler', how='outer')\
                        .merge(wickets, on='bowler', how='outer')\
                        .merge(four_wickets, on='bowler', how='outer')\
                        .merge(five_wickets, on='bowler', how='outer').fillna(0)

In [113]:
# Check the new dataframe

bowling_stats_merged.head()

,bowler,bowler_innings,balls_bowled,runs_conceded,wickets,four_wickets,five_wickets
0,a ashish reddy,20,262,396,18.0,0.0,0.0
1,a badoni,2,12,11,2.0,0.0,0.0
2,a chandila,12,234,242,11.0,1.0,0.0
3,a choudhary,5,101,144,5.0,0.0,0.0
4,a dananjaya,1,24,47,0.0,0.0,0.0


In [114]:
# Calculate bowling average score - runs_conceded / wickets

bowling_stats_merged['bowling_average'] = round((bowling_stats_merged['runs_conceded'] / bowling_stats_merged['wickets']), 2).fillna(0)

# Replace infinite values with 0

bowling_stats_merged['bowling_average'] = np.where(bowling_stats_merged\
                                        ['bowling_average'] == np.inf, 0, bowling_stats_merged['bowling_average'])

In [115]:
# Calculate bowling economy rate - runs_conceded / (balls_bowled / 6)

bowling_stats_merged['bowling_economy_rate'] = round((bowling_stats_merged['runs_conceded']/(bowling_stats_merged['balls_bowled'] / 6)), 2)

In [116]:
# Calculate bowling strike rate - balls_bowled / wickets

bowling_stats_merged['bowling_strike_rate'] = round((bowling_stats_merged['balls_bowled']/bowling_stats_merged['wickets']), 2)

# Replace infinite values with 0

bowling_stats_merged['bowling_strike_rate'] = np.where(bowling_stats_merged\
                                        ['bowling_strike_rate'] == np.inf, 0, bowling_stats_merged['bowling_strike_rate'])

In [117]:
bowling_stats_merged

,bowler,bowler_innings,balls_bowled,runs_conceded,wickets,four_wickets,five_wickets,bowling_average,bowling_economy_rate,bowling_strike_rate
0,a ashish reddy,20,262,396,18.0,0.0,0.0,22.00,9.07,14.56
1,a badoni,2,12,11,2.0,0.0,0.0,5.50,5.50,6.00
2,a chandila,12,234,242,11.0,1.0,0.0,22.00,6.21,21.27
3,a choudhary,5,101,144,5.0,0.0,0.0,28.80,8.55,20.20
4,a dananjaya,1,24,47,0.0,0.0,0.0,0.00,11.75,0.00
...,...,...,...,...,...,...,...,...,...,...
467,ybk jaiswal,1,1,6,0.0,0.0,0.0,0.00,36.00,0.00
468,yk pathan,82,1147,1415,42.0,0.0,0.0,33.69,7.40,27.31
469,ys chahal,130,2856,3624,166.0,3.0,1.0,21.83,7.61,17.20
470,yuvraj singh,73,869,1077,36.0,2.0,0.0,29.92,7.44,24.14


In [119]:
# Export

# bowling_stats_merged.to_csv('~/code/patrickevans29/raw_data/bowling_features_dataset.csv', index=False)

# 3. Merging DataFrames

### Combining the new dataframes to include all player data

In [122]:
# Rename and reassign dataframe variables

player_bat = batting_stats_merged.rename(columns={'batter': 'player'})
player_bowl = bowling_stats_merged.rename(columns={'bowler': 'player'})

In [126]:
# Merge and fill nan values as 0

player_dataset = player_bat.merge(player_bowl, on='player', how='outer').fillna(0)

In [127]:
# View the final dataset

player_dataset

,player,bat_innings,total_runs,not_out,zero,50s,100s,fours,sixes,high_score,...,batting_strike_rate,bowler_innings,balls_bowled,runs_conceded,wickets,four_wickets,five_wickets,bowling_average,bowling_economy_rate,bowling_strike_rate
0,a ashish reddy,23.0,280.0,8.0,0.0,0.0,0.0,16.0,15.0,36.0,...,145.08,20.0,262.0,396.0,18.0,0.0,0.0,22.0,9.07,14.56
1,a badoni,11.0,161.0,3.0,1.0,1.0,0.0,11.0,7.0,54.0,...,123.85,2.0,12.0,11.0,2.0,0.0,0.0,5.5,5.50,6.00
2,a chandila,2.0,4.0,2.0,1.0,0.0,0.0,0.0,0.0,4.0,...,57.14,12.0,234.0,242.0,11.0,1.0,0.0,22.0,6.21,21.27
3,a chopra,6.0,53.0,0.0,0.0,0.0,0.0,7.0,0.0,24.0,...,74.65,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
4,a choudhary,3.0,25.0,2.0,0.0,0.0,0.0,1.0,1.0,15.0,...,125.00,5.0,101.0,144.0,5.0,0.0,0.0,28.8,8.55,20.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647,sm boland,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,2.0,42.0,54.0,2.0,0.0,0.0,27.0,7.71,21.00
648,ss agarwal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,1.0,24.0,42.0,1.0,0.0,0.0,42.0,10.50,24.00
649,ss mundhe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,1.0,6.0,6.0,1.0,0.0,0.0,6.0,6.00,6.00
650,ss sarkar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,2.0,25.0,34.0,1.0,0.0,0.0,34.0,8.16,25.00


In [136]:
# Export

# player_dataset.to_csv('~/code/patrickevans29/raw_data/player_dataset.csv', index=False)

In [162]:
df.head()

,ID,innings,overs,ballnumber,batter,bowler,non_striker,extra_type,batsman_run,extras_run,...,TossDecision,SuperOver,WinningTeam,WonBy,Margin,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,1312200,1,1,1,ybk jaiswal,mohammed shami,jc buttler,noextra,0,0,...,bat,n,gujarat titans,wickets,7.0,hh pandya,"['ybk jaiswal', 'jc buttler', 'sv samson', 'd ...","['wp saha', 'shubman gill', 'ms wade', 'hh pan...",cb gaffaney,nitin menon
1,1312200,1,1,2,ybk jaiswal,mohammed shami,jc buttler,legbyes,0,1,...,bat,n,gujarat titans,wickets,7.0,hh pandya,"['ybk jaiswal', 'jc buttler', 'sv samson', 'd ...","['wp saha', 'shubman gill', 'ms wade', 'hh pan...",cb gaffaney,nitin menon
2,1312200,1,1,3,jc buttler,mohammed shami,ybk jaiswal,noextra,1,0,...,bat,n,gujarat titans,wickets,7.0,hh pandya,"['ybk jaiswal', 'jc buttler', 'sv samson', 'd ...","['wp saha', 'shubman gill', 'ms wade', 'hh pan...",cb gaffaney,nitin menon
3,1312200,1,1,4,ybk jaiswal,mohammed shami,jc buttler,noextra,0,0,...,bat,n,gujarat titans,wickets,7.0,hh pandya,"['ybk jaiswal', 'jc buttler', 'sv samson', 'd ...","['wp saha', 'shubman gill', 'ms wade', 'hh pan...",cb gaffaney,nitin menon
4,1312200,1,1,5,ybk jaiswal,mohammed shami,jc buttler,noextra,0,0,...,bat,n,gujarat titans,wickets,7.0,hh pandya,"['ybk jaiswal', 'jc buttler', 'sv samson', 'd ...","['wp saha', 'shubman gill', 'ms wade', 'hh pan...",cb gaffaney,nitin menon


In [139]:
df.columns

Index(['ID', 'innings', 'overs', 'ballnumber', 'batter', 'bowler',
       'non_striker', 'extra_type', 'batsman_run', 'extras_run', 'total_run',
       'non_boundary', 'isWicketDelivery', 'player_out', 'kind',
       'fielders_involved', 'BattingTeam', 'City', 'Date', 'Season',
       'MatchNumber', 'Team1', 'Team2', 'Venue', 'TossWinner', 'TossDecision',
       'SuperOver', 'WinningTeam', 'WonBy', 'Margin', 'Player_of_Match',
       'Team1Players', 'Team2Players', 'Umpire1', 'Umpire2'],
      dtype='object')

In [148]:
df[df['TossWinner'] == df['WinningTeam']].groupby('ID', as_index=False)['ID'].unique().shape[0] / df.groupby('ID', as_index=False)['ID'].unique().shape[0]

0.5147368421052632

In [193]:
df2 = pd.read_csv('~/code/patrickevans29/raw_data/complete_cleaned_dataset.csv', converters={'Team1Players': pd.eval, 'Team2Players':pd.eval})

In [238]:
temp_group = df2.groupby(['ID', 'Player_of_Match'], as_index=False).count()
player_of_match = temp_group.groupby(['Player_of_Match'], as_index=False)['ID'].count().rename(columns={'ID': 'Player_of_Match', 'Player_of_Match': 'player'})

In [243]:
for team in df2['WinningTeam']:
    if team == df2['Team1']:
        df2['Winning_list'] = df2['Team1Players']
    elif team == df2['Team2']:
        df2['Winning_list'] = df2['Team2Players']

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [253]:
for team in df2['WinningTeam']:
    if lambda row: row['Team1'] == team:
        row['winning_list'] = row['Team1Players']
    elif lambda row: row['Team2'] == team:
        row['winning_list'] = row['Team2Players']

NameError: name 'row' is not defined